# Embeddings

## Environment

In [ ]:
!pip install sentence_transformers tiktoken openai -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 15.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import pyarrow

import tiktoken
import openai
from openai.embeddings_utils import get_embedding

from sentence_transformers import SentenceTransformer
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/semantic_song_search

Mounted at /content/gdrive
/content/gdrive/MyDrive/Projects/semantic_song_search


## Load Data

In [ ]:
df = pd.read_parquet('data/5mil_song_set_clean_v2_good_artists.parquet')
print(df.shape)
df.info()

(17000, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   title              17000 non-null  object  
 1   tag                17000 non-null  category
 2   artist             17000 non-null  object  
 3   year               17000 non-null  int16   
 4   views              17000 non-null  int32   
 5   lyrics             17000 non-null  object  
 6   normalized_lyrics  17000 non-null  object  
dtypes: category(1), int16(1), int32(1), object(4)
memory usage: 647.8+ KB


In [ ]:
pd.set_option('max_colwidth', 500)
df.sample(5).T

,6401,16315,11472,14603,16287
title,Life Is a Gamble,This Is Hip 1991,The Stars Keep on Calling My Name Demo,The Unicorns: 2014 Demo Version,Raining In My Heart
tag,rap,pop,rock,pop,pop
artist,Marvin Gaye,John Lee Hooker,Mac DeMarco,The Unicorns,Ringo Starr
year,1972,1991,2013,2002,2015
views,1197,107,288,185,19
lyrics,"Life is a gamble, oh, oh baby\nWhere you win or lose\nI'm right, just paid my dues",This is hip pretty baby\nThis is hip pretty baby\nThis is hip pretty baby\nI messed around and fell in love\n\nYou know you called me up on your telephone\nYou want me to take you for a ride\nThis is hip pretty baby\nThis is hip pretty baby\nThis is hip pretty baby\nI messed around and fell in love\n\nWill you come to my house and you dance with me\nYou hold me tight and you kiss me too\nThis is hip pretty baby\nThis is hip pretty baby\nThis is hip pretty baby\nYou gonna kiss me baby\nI mess...,"\nHoney, the stars keep on calling my name\nBut don't worry, I've told you again and again\nWhen I'm down, you're always the first one to know\nSkipping town, I'll take you wherever I go\n\n\nAnd I just wanna go\nI just wanna go\n\n\nHoney, this town’s really taking it’s toll\nBut don’t worry, I’m all through with feeling this old\nWhen I’m down, you’re always the first on my mind\nSkipping town, let’s get out and see what we find\n\n\nAnd I just wanna go\nI just wanna go\n\n\nHoney, the sta...","Tomorrow is 2014\nWhen I'll be 32\nAnd we'll be 13\nBut will there be enough room for me?\nEnough oxygen, for me to breathe\n\nOh yeah oh yeah\n\nI looked in into my crystal ball\nSee gummies in the sunny\nRiding moonbeams into money\nBut they're the guilty phase\nGluttony just doesn't pay\nSo volunteer your journey\nIt's good publicity\n\nDa da da da da da da\n\nIn full force\nIn full stride\nSide by side 'till 2025\nIn full force\nIn full stride\nSide by side 'till 2025\nLie lie lie lie li...","They're gonna put me in the movies\nThey're gonna make a big star out of me\nWe'll make a film about a man that's sad and lonely\nAnd all I gotta do is act naturally\nWell, I bet you I'm gonna be a big star\nMight win an 'Oscar', you can never tell\nThe movie's gonna make me a big star\n'Cause I can play the part so well\nWell, I hope you'll come and see me in the movies\nThen I'll know that you will plainly see\nThe biggest fool that ever hit the big time\nAnd all I gotta do is act naturall..."
normalized_lyrics,life gamble oh oh baby win lose right paid due,hip pretty baby hip pretty baby hip pretty baby mess around fell love know call telephone want take ride hip pretty baby hip pretty baby hip pretty baby mess around fell love come house dance hold tight kiss hip pretty baby hip pretty baby hip pretty baby gon na kiss baby mess around fell love ooh yeah call talk long time talk telephone tellin comin round baby hip pretty baby hip pretty baby hip pretty baby mess around fell love blast rock rock soul set fire hip pretty baby hip pretty baby h...,honey star keep call name worry tell always first one know skip town take wherever go wan na go wan na go honey town really take toll worry feel old always first mind skip town let get see find wan na go wan na go honey star keep call name worry tell always first one know skip town take wherever go wan na go wan na go wan na go wan na go kick rick,tomorrow enough room enough oxygen breathe oh yeah oh yeah look crystal ball see gummies sunny rid moonbeam money guilty phase gluttony pay volunteer journey good publicity da da da da da da da full force full stride side side till full force full stride side side till lie lie lie lie lie lie lie lie lie,gon na put movie gon na make big star make film man sad lonely get ta act naturally well bet gon na big star might win oscar never tell movie gon na make big star cause play part well well hope come see movie know plainly see big fool ever hit big time get ta act naturally make film man sad lonely beggin upon bend knee play part need rehearsin get ta act naturally 

In [ ]:
df['word_count'] = df['lyrics'].apply(lambda x: len(x.split()))

## Sentence Transformers

In [ ]:
# download models

#model_name = 'all-MiniLM-L12-v2'
#model = SentenceTransformer(model_name)
#model.save('./' + model_name)

#model_name = 'all-distilroberta-v1'
#model = SentenceTransformer(model_name)
#model.save('./' + model_name)

In [ ]:
# sentence transformer models

minilm = SentenceTransformer('./all-MiniLM-L12-v2')
minilm = minilm.to(torch.device('cuda')) # use GPU

roberta = SentenceTransformer('./all-distilroberta-v1')
roberta = roberta.to(torch.device('cuda')) # use GPU


In [ ]:
%%time
embeddings_minilm = minilm.encode(df['lyrics'])

CPU times: user 34.7 s, sys: 1.25 s, total: 35.9 s
Wall time: 14.7 s


In [ ]:
%%time
embeddings_roberta = roberta.encode(df['lyrics'])

CPU times: user 1min 1s, sys: 1.36 s, total: 1min 3s
Wall time: 41.1 s


In [ ]:
# append
df['embedding_minilm'] = list(embeddings_minilm)
df['embedding_roberta'] = list(embeddings_roberta)

## OpenAI Text Embeddings

In [ ]:
# openai api config
with open("private/openai_secret_key.txt", "r") as f:
    my_little_secret = f.read()
openai.api_key = my_little_secret

In [ ]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"

In [ ]:
# check if any songs are too long
encoding = tiktoken.get_encoding("cl100k_base") # encoding for text-embedding-ada-002
max_tokens = 8191  # text-embedding-ada-002 max
sum(df.lyrics.apply(lambda x: len(encoding.encode(x))) > max_tokens)

0

In [ ]:
%%time

from tqdm import tqdm
from tqdm.auto import tqdm
tqdm.pandas()

df["embedding_gpt"] = df.lyrics.progress_apply(lambda x: get_embedding(x, engine=embedding_model))

  0%|          | 0/17000 [00:00<?, ?it/s]

CPU times: user 1min 24s, sys: 10.3 s, total: 1min 35s
Wall time: 1h 4min 31s


### GloVe

In [ ]:
# download model
#model_name = 'average_word_embeddings_glove.840B.300d'
#model = SentenceTransformer(model_name)
#model.save('./' + model_name)

In [ ]:
glove = SentenceTransformer('./average_word_embeddings_glove.840B.300d')

In [ ]:
%%time
# compute vector representations of the comments
embeddings_glove = glove.encode(df['normalized_lyrics'])

CPU times: user 2.46 s, sys: 123 ms, total: 2.58 s
Wall time: 2.58 s


In [ ]:
df['embedding_glove'] = list(embeddings_glove)

### Save for Later

In [ ]:
df.to_parquet('data/v2ga_w_embeddings.parquet')